## Configurable Decorators: Decorators with Arguments

- A basic decorator adds fixed behavior; sometimes you need to *configure* that behaviour (e.g. how many retries, which log level).  
- You cannot pass options directly to a plain `@decorator`, because that decorator receives only the target function.  
- Solution: call a *factory* that takes options and returns a decorator, then apply it with `@factory(option=value)`.  

In [ ]:
def timing_decorator(original_function):
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = original_function(*args, **kwargs)
        duration = time.perf_counter() - start
        print(f"{original_function.__name__} took {duration:.3f}s")

        return result

    return wrapper

## The Decorator Factory Pattern

- **Factory function** receives configuration arguments and returns the **actual decorator**.  
- The actual **decorator** still takes the target function and builds a **wrapper**.  
- The wrapper can access both the factory’s configuration (via a closure) and the call‑time `*args / **kwargs` for the target function.  
- Three nested layers keep concerns separated: configuration ➜ decoration ➜ runtime.

### Applying Decorators with Arguments

- Use `@factory(arg1, arg2…)` above the function definition.  
- At *definition* time Python calls the factory, gets back a decorator, and applies that decorator to the function.  
- Callers of the function automatically get the behaviour configured by the factory.

## Example: Retry Decorator Factory

- A practical DevOps scenario: retry a flaky operation a configurable number of times.  
- The factory takes `max_attempts`; the wrapper loops until success or until attempts are exhausted, re‑raising the last error.  

In [19]:
import random

def retry(max_attempts=3):
    def decorator(func):
        def wrapper(*args, **kwargs):
            for attempt in range(1, max_attempts + 1):
                try:
                    print(f"Attempt {attempt}/{max_attempts}")
                    return func(*args, **kwargs)
                except Exception as e:
                    print(f" Error: {e}")
                    if attempt == max_attempts:
                        raise

        return wrapper
    return decorator

@retry(4)
def sometimes_fails():
    if random.random() < 0.7:
        raise RuntimeError("Flaky failure")
    return "Success!"

print(f"Result: {sometimes_fails()}")

Attempt 1/4
Result: Success!
